In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
train_values = pd.read_csv('train_values.csv', dtype= {'building_id': np.int32,\
                                              'geo_level_1_id': np.int8,\
                                              'geo_level_2_id': np.int16,\
                                              'geo_level_3_id': np.int16,\
                                              'count_floors_pre_eq': np.int16,\
                                              'age': np.int16,\
                                              'area_percentage': np.int8,\
                                              'height_percentage': np.int8,\
                                              'land_surface_condition': 'category',\
                                              'foundation_type': 'category',\
                                              'roof_type': 'category',\
                                              'ground_floor_type':'category',\
                                              'other_floor_type': 'category',\
                                              'position': 'category',\
                                              'plan_configuration':'category',\
                                              'has_superstructure_adobe_mud':'boolean',\
                                              'has_superstructure_mud_mortar_stone':'boolean',\
                                              'has_superstructure_stone_flag':'boolean',\
                                              'has_superstructure_cement_mortar_stone':'boolean',\
                                              'has_superstructure_mud_mortar_brick':'boolean',\
                                              'has_superstructure_cement_mortar_brick':'boolean',\
                                              'has_superstructure_timber':'boolean',\
                                              'has_superstructure_bamboo':'boolean',\
                                              'has_superstructure_rc_non_engineered':'boolean',\
                                              'has_superstructure_rc_engineered':'boolean',\
                                              'has_superstructure_other':'boolean',\
                                              'legal_ownership_status':'category',\
                                              'count_families': np.int16,\
                                              'has_secondary_use':'boolean',\
                                              'has_secondary_use_agriculture':'boolean',\
                                              'has_secondary_use_hotel':'boolean',\
                                              'has_secondary_use_rental':'boolean',\
                                              'has_secondary_use_institution':'boolean',\
                                              'has_secondary_use_school':'boolean',\
                                              'has_secondary_use_industry':'boolean',\
                                              'has_secondary_use_health_post':'boolean',\
                                              'has_secondary_use_gov_office':'boolean',\
                                              'has_secondary_use_use_police':'boolean',\
                                              'has_secondary_use_other':'boolean'
                                              })
train_labels = pd.read_csv("train_labels.csv")
test_values = pd.read_csv("test_values.csv")

In [4]:
train = train_values.merge(train_labels,on = "building_id")

In [5]:
x_pre = pd.get_dummies(train_values).set_index('building_id')
y_pre = train_labels.loc[:,'damage_grade']

x = x_pre
y = y_pre
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [6]:
hiper = {'n_estimators': [100, 250, 500, 1000], 
'min_samples_split':[20, 50, 250, 500]}
rf_clf = RandomForestClassifier()
gd_sr = GridSearchCV(rf_clf,param_grid=hiper,scoring='f1_micro',cv=5,n_jobs=-1)
gd_sr.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'min_samples_split': [20, 50, 250, 500],
                         'n_estimators': [100, 250, 500, 1000]},
             scoring='f1_micro')

In [10]:
gd_sr.best_score_


0.7194646968534152

In [11]:
gd_sr.best_params_

{'min_samples_split': 20, 'n_estimators': 500}

In [12]:
rf = RandomForestClassifier(min_samples_split= 20, n_estimators=500)
model = rf.fit(x_train, y_train)

model.predict(x_test)
print("Training Score: {}".format(rf.score(x_train, y_train)))
print("Test Score: {}".format(rf.score(x_test, y_test)))

Training Score: 0.8246834228702993
Test Score: 0.7236238752134456


In [13]:
test_values_subset = test_values.set_index('building_id')
test_values_subset = pd.get_dummies(test_values_subset)
#segundo submission
predictions = model.predict(test_values_subset)
test_values_subset['damage_grade'] = predictions
test_values_subset.loc[:,'damage_grade'].to_csv("submission_2.csv")

In [18]:
from sklearn.inspection import permutation_importance
import time as time

start_time = time.time()
result = permutation_importance(
    model, x_test, y_test, n_repeats=10, random_state=42, n_jobs=2)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: "
      f"{elapsed_time:.3f} seconds")

forest_importances = pd.Series(result.importances_mean, index=feature_names)


PicklingError: Could not pickle the task to send it to the workers.